# Install and import libraries

In [1]:
!pip install einops
!pip install datasets
!pip install torch-fidelity
!pip install torchmetrics
!pip3 install image-similarity-measures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 435 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 9.0 MB/s 
     |████████████████████████████████| 212 kB 47.4 MB/s 
     |████████████████████████████████| 182 kB 52.4 MB/s 
     |████████████████████████████████| 115 kB 46.4 MB/s 
     |████████████████████████████████| 127 kB 47.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 35.2 MB/s 
Looking in indexes: https://pypi.org/simpl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import numpy
from numpy import cov, trace, iscomplexobj
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import sys

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Paths (Checkpoint)

In [2]:
FashionCNN_checkpoint = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/fashion_mnist_CNNClassifier_91Acc.pt"
DiffusionModel_checkpoint = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/FashionMNIST/checkpoint/checkpoint_T500/fashion_mnist/Unet.ckpt"

Stats_path = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/FashionMNIST/checkpoint/checkpoint_T500/fashion_mnist/stats_T500.csv"

# Load FashionMNIST Dataset to get Real Images

In [3]:
train_set = torchvision.datasets.FashionMNIST("./data", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=100, shuffle=False)

# Generate Images from trained Diffusion Model

In [6]:
!git clone https://github.com/GangaMegha/Diffusion-Models.git 

Cloning into 'Diffusion-Models'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 173 (delta 95), reused 127 (delta 53), pack-reused 0
Receiving objects: 100% (173/173), 29.34 KiB | 9.78 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [4]:
%cd /content/Diffusion-Models/src
!ls

/content/Diffusion-Models/src
config.py  data_loader.py  logger.py  models	   train
data	   diffusion	   main.py    __pycache__  visualize


#### Load Model

In [5]:

from models.unet import Unet as Model
from config import LEVEL, TRAIN, MODEL, MODEL_TYPE, DATASET_NAME, CONFIG, CHECKPOINT_PATH, RESULT_PATH, LOG_PATH

from diffusion.reverse_data_generate import sample, p_sample
from diffusion.var_schedule import alpha_beta

from data_loader import reverse_transform

cfg = CONFIG[MODEL_TYPE.value][DATASET_NAME]
Noise_model = Model(cfg)
Noise_model_checkpoint = torch.load(DiffusionModel_checkpoint)
Noise_model.load_state_dict(Noise_model_checkpoint['state_dict'])
Noise_model.to(device)
Noise_model.eval()

variance_dict = alpha_beta(cfg.get('T'), schedule=cfg.get("var_schedule"))

#### Interpolation

In [120]:
from tqdm.auto import tqdm
from torchvision.utils import save_image

from diffusion.forward_diffusion import get_noisy_image

it = iter(train_loader)
batch = next(it)[0]

# img1 = get_noisy_image(batch[3], torch.tensor([cfg.get('T')-1]), variance_dict)[None, None, :, :]/255.0
# img2 = get_noisy_image(batch[10], torch.tensor([cfg.get('T')-1]), variance_dict)[None, None, :, :]/255.0

img1 = get_noisy_image(2*batch[55]-1, torch.tensor([499]), variance_dict)[None, None, :, :]/255.0
img2 = get_noisy_image(2*batch[44]-1, torch.tensor([499]), variance_dict)[None, None, :, :]/255.0
# img2 = get_noisy_image(batch[-11], torch.tensor([100]), variance_dict)[None, None, :, :]/255.0

# plt.imshow((2*img1-1).reshape(28,28), cmap='gray')
img1 = img1*2 - 1
img2 = img2*2 - 1

imgs = []
for i in np.linspace(0, 1, 9):
  print(i)
  imgs.append(i*img1 + (1-i)*img2)
  
imgs = torch.cat(imgs, dim=0).to(device)
b = len(imgs)

for i in tqdm(reversed(range(0, 500)), desc='sampling loop time step', total=500):
    imgs = p_sample(Noise_model, imgs, torch.full((b,), i, device=device, dtype=torch.long), i, variance_dict)

save_image((imgs+1)*0.5, "FashionMNIST_T500_Interpolation.png", nrow=16)

0.0
0.125
0.25
0.375
0.5
0.625
0.75
0.875
1.0


sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

In [15]:
from tqdm.auto import tqdm
from torchvision.utils import save_image

img1 = torch.randn((1, 1, 28, 28), device=device)
img2 = torch.randn((1, 1, 28, 28), device=device)

imgs = []
for i in np.linspace(0,1, 9):
  print(i)
  imgs.append(i*img1 + (1-i)*img2)
  
imgs = torch.cat(imgs, dim=0)
b = len(imgs)

for i in tqdm(reversed(range(0, 500)), desc='sampling loop time step', total=500):
    imgs = p_sample(Noise_model, imgs, torch.full((b,), i, device=device, dtype=torch.long), i, variance_dict)

save_image((imgs+1)*0.5, "FashionMNIST_T500_Interpolation.png", nrow=16)

0.0
0.125
0.25
0.375
0.5
0.625
0.75
0.875
1.0


sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

#### Generate Images

In [ ]:
fake_images = []
for i in range(10):
  fake_images_list = sample(Noise_model, variance_dict, cfg, sample_cnt=100, every50=False)
  fake_images.append(reverse_transform()(torch.cat(fake_images_list, dim=0)))

# Get FashionCNN results for Real and Fake Images

In [ ]:
class FashionCNN(nn.Module):
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding="same"),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding="same"),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*7*7, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = nn.ReLU()(self.fc1(out))
        out = nn.Softmax()(self.fc2(out))
        
        return out

FashionCNN_model = FashionCNN()
CNN_checkpoint = torch.load(FashionCNN_checkpoint)
FashionCNN_model.load_state_dict(CNN_checkpoint['state_dict'])
FashionCNN_model.to(device)
FashionCNN_model.eval()

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.clone().detach()
    return hook

FashionCNN_model.fc1.register_forward_hook(get_activation('fc1'))

#### Real Images

In [ ]:
real_out = []
output_real = []
it = iter(train_loader)
batches = []
for i in range(10):
  batch = next(it)
  batches.append(batch[0])
  output_real.append(FashionCNN_model(batch[0].to(device)))
  real_out.append(activation['fc1'].detach().cpu().numpy())

#### Fake Images

In [ ]:
fake_out = []
output_fake = []
for i in range(10):
  output_fake.append(FashionCNN_model((fake_images[i].to(dtype=torch.float)/255.0).to(device)))
  fake_out.append(activation['fc1'].detach().cpu().numpy())

## Calculate FID

In [ ]:
# Ref : https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/
def calculate_fid(act1, act2):
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [ ]:
fid_cnn_list = []
for i in range(10):
  fid_cnn_list.append(calculate_fid(real_out[i], fake_out[i]))

print("fid_cnn_list : ", fid_cnn_list)
print("Mean : ", np.mean(np.array(fid_cnn_list)))
print("Std : ", np.std(np.array(fid_cnn_list)))

## Calculate Inception Score using CNN Classifier Probabilities

In [ ]:
# Ref : https://machinelearningmastery.com/how-to-implement-the-inception-score-from-scratch-for-evaluating-generated-images/
# calculate the inception score for p(y|x)
def calculate_inception_score(p_yx, eps=1E-16):
	# calculate p(y)
	p_y = np.expand_dims(p_yx.mean(axis=0), 0)
	# kl divergence for each image
	kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
	# sum over classes
	sum_kl_d = kl_d.sum(axis=1)
	# average over images
	avg_kl_d = np.mean(sum_kl_d)
	# undo the logs
	is_score = np.exp(avg_kl_d)
	return is_score

In [ ]:
is_cnn_list = []
for i in range(10):
  is_cnn_list.append(calculate_inception_score(output_fake[i].detach().cpu().numpy()))

print("is_cnn_list : ", is_cnn_list)
print("Mean : ", np.mean(np.array(is_cnn_list)))
print("Std : ", np.std(np.array(is_cnn_list)))

## Calculate SSIM and RMSE

In [ ]:
from image_similarity_measures.quality_metrics import rmse, ssim, sre, uiq

ssim_list_class = []
rmse_list_class = []

for label in range(10):
  ssim_list = []
  rmse_list = []

  for idx in range(10):
    predicted_real = torch.max(output_real[idx], 1)[1]
    predicted_fake = torch.max(output_fake[idx], 1)[1]

    real_img = []
    fake_img = []
    for j in range(len(predicted_real)):
      if predicted_real[j]==label:
        real_img.append(batches[idx][j])
    for j in range(len(predicted_fake)):
      if predicted_fake[j]==label:
        fake_img.append((fake_images[idx][j].to(dtype=torch.float)/255.0).to(device))

    for j in range(len(real_img)):
      for k in range(len(fake_img)):
        ssim_list.append(ssim(real_img[j].permute(1, 2, 0).cpu().numpy(), fake_img[k].permute(1, 2, 0).cpu().numpy()))
        rmse_list.append(rmse(real_img[j].permute(1, 2, 0).cpu().numpy(), fake_img[k].permute(1, 2, 0).cpu().numpy()))

  ssim_list_class.append(np.mean(np.array(ssim_list)))
  rmse_list_class.append(np.mean(np.array(rmse_list)))

print("ssim_list_class : ", ssim_list_class)
print("rmse_list_class : ", rmse_list_class)

ssim_list_class = np.nan_to_num(np.array(ssim_list_class), nan=0)
rmse_list_class = np.nan_to_num(np.array(rmse_list_class), nan=1)

print("ssim_list_class : ", ssim_list_class)
print("rmse_list_class : ", rmse_list_class)

print("ssim Mean : ", np.mean(ssim_list_class))
print("ssim Std : ", np.std(ssim_list_class))

print("rmse Mean : ", np.mean(rmse_list_class))
print("rmse Std : ", np.std(rmse_list_class))

ssim_list_class :  [0.9999532560229581, 0.9999653231001326, 0.9999529324990983, 0.9999621573417958, 0.9999600030414929, 0.999951818798513, 0.9999439974789275, 0.9999738826443573, 0.9999313274782419, 0.9999664544318044]
rmse_list_class :  [8.7888104e-05, 7.7351026e-05, 9.201101e-05, 7.884341e-05, 8.2812265e-05, 8.778537e-05, 9.6386546e-05, 6.9133195e-05, 0.0001118533, 8.1303806e-05]
ssim_list_class :  [0.99995326 0.99996532 0.99995293 0.99996216 0.99996    0.99995182
 0.999944   0.99997388 0.99993133 0.99996645]
rmse_list_class :  [8.7888104e-05 7.7351026e-05 9.2011011e-05 7.8843412e-05 8.2812265e-05
 8.7785367e-05 9.6386546e-05 6.9133195e-05 1.1185330e-04 8.1303806e-05]
ssim Mean :  0.9999561152837323
ssim Std :  1.1649136728123086e-05
rmse Mean :  8.6536806e-05
rmse Std :  1.122053e-05


# Compute True FID and IS using Inception Network

In [ ]:
from train.metrics import FID, IS

In [ ]:
FID_ = FID().to(device)
IS_ = IS().to(device)

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth


  0%|          | 0.00/91.2M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [ ]:
it = iter(train_loader)
FID_list, IS_list = [], []

for i in range(10):
  batch = next(it)[0]
  FID_list.append(FID_(fake_images[i].to(device), (batch*255).to(dtype=torch.uint8).to(device), cfg["grayscale"]).item())
  IS_list.append(IS_(fake_images[i].to(device), cfg["grayscale"]).item())

print("FID_list : ", FID_list)
print("IS_list : ", IS_list)

print("FID Mean : ", np.mean(np.array(FID_list)))
print("FID Std : ", np.std(np.array(FID_list)))

print("IS Mean : ", np.mean(np.array(IS_list)))
print("IS Std : ", np.std(np.array(IS_list)))

FID_list :  [145.5977325439453, 146.1687469482422, 143.8839569091797, 147.5160675048828, 148.0241241455078, 148.95191955566406, 148.58302307128906, 153.00155639648438, 160.1544647216797, 141.3311767578125]
IS_list :  [3.1128711700439453, 3.277529239654541, 3.090423345565796, 3.162557363510132, 3.0484530925750732, 3.0440945625305176, 3.2014527320861816, 2.929187774658203, 3.320741653442383, 3.3122475147247314]
FID Mean :  148.32127685546874
FID Std :  4.936948663765205
IS Mean :  3.1499558448791505
IS Std :  0.12237623521877555


# Save Computed stats

In [ ]:
# load csv module
import csv

# define a dictionary with key value pairs
diction = {
    "Metric" : "Value",
    "FID Mean" : np.mean(np.array(FID_list)),
    "FID Std" : np.std(np.array(FID_list)),
    "IS Mean" : np.mean(np.array(IS_list)),
    "IS Std" : np.std(np.array(IS_list)),
    "SSIM Mean" : np.mean(np.array(ssim_list_class)),
    "SSIM Std" : np.std(np.array(ssim_list_class)),
    "RMSE Mean" : np.mean(np.array(rmse_list_class)),
    "RMSE Std" : np.std(np.array(rmse_list_class)),
    "FID CNN Mean" : np.mean(np.array(fid_cnn_list)),
    "FID CNN Std" : np.std(np.array(fid_cnn_list)),
    "IS CNN Mean" : np.mean(np.array(is_cnn_list)),
    "IS CNN Std" : np.std(np.array(is_cnn_list))
    }

# open file for writing, "w" is writing
w = csv.writer(open(Stats_path, "w"))

# loop over dictionary keys and values
for key, val in diction.items():

    # write every key and value to file
    w.writerow([key, val])


# Visualize Generated Images

In [ ]:
from torchvision.utils import save_image
all_images = torch.cat(fake_images_list, dim=0)
save_image((all_images[:16*]+1)*0.5, "FashionMNIST_T500_FakeImages.png", nrow=16)